In [1]:

!pip install sumonet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 787.9/787.9 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 7.9 MB/s eta 0:00:00
  Created wheel for sumonet: filename=sumonet-0.1.5-py3-none-any.whl size=790946 sha256=5773b447a4424980d4691b9a686314d79e1dedd29cd280306f3b00699e892237
  Stored in directory: /root/.cache/pip/wheels/5e/53/5a/dbf0d69cf2495a03cde7d25de014f657571b6617a6bbf64bdc
  Created wheel for epitopepredict: filename=epitopepredict-0.5.0-py3-none-any.whl size=5973737 sha256=95e7c0db760cafa01682b038438381041db508719e94756c3258b3ae73e60925
  Stored in directory: /root/.cache/pip/wheels/e3/d6/36/567730c38eb067fba937ec0b48e2d508d0e171b9a20b7cc598
Successfully built sumonet epitopepredict


In [37]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load your CSV file
file_path = 'data.csv'  # Replace with the actual file path
df = pd.read_csv(file_path)

# Assuming 'Sequence' is the column containing protein sequences, and 'Label' is the target column
X = df['Sequence'].tolist()  # Convert 'Sequence' column to a list
y = df['Label'].tolist()  # Convert 'Label' column to a list

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [38]:
from sumonet.utils.encodings import Encoding
encoder = Encoding(encoderType='one-hot')
X_train_encoded = encoder.encode_data(X_train)

In [39]:
import numpy as np
y_train = np.asarray(y_train)
y_train = (y_train[:,None] == np.arange(2)).astype(int)

In [41]:
from sumonet.model.architecture import SUMOnet
model = SUMOnet(input_shape = X_train_encoded.shape[1:] )
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
model.fit(X_train_encoded,y_train,epochs=3)

Epoch 1/3
187/187 [==============================] - 15s 48ms/step - loss: 0.7019 - accuracy: 0.5097
Epoch 2/3
187/187 [==============================] - 10s 52ms/step - loss: 0.6951 - accuracy: 0.5248
Epoch 3/3
187/187 [==============================] - 10s 55ms/step - loss: 0.6952 - accuracy: 0.5069


In [42]:
encoder = Encoding(encoderType='one-hot')
X_test_encoded = encoder.encode_data(X_test)
y_preds = model.predict(X_test_encoded)

47/47 [==============================] - 2s 15ms/step


In [43]:
from sklearn import metrics

def f1_score(y_true, y_probs):
    return metrics.f1_score(y_true, y_probs.argmax(-1))

def mcc(y_true, y_probs):
    return metrics.matthews_corrcoef(y_true, y_probs.argmax(-1))

def rocAuc_score(y_true, y_probs):
    lr_fpr, lr_tpr, _ = metrics.roc_curve(y_true, y_probs[:,1])
    return metrics.auc(lr_fpr, lr_tpr)

def aupr_score(y_true, y_probs):
    precision, recall, _ = metrics.precision_recall_curve(y_true, y_probs[:,1])
    return metrics.auc(recall, precision)

def accuracy(y_true, y_probs):
    predicted_labels = y_probs.argmax(-1)
    return metrics.accuracy_score(y_true, predicted_labels)

def auc_score(y_true, y_probs):
    return metrics.roc_auc_score(y_true, y_probs[:,1])

def evaluate(y_true, y_probs, metrics_):
    metricsDict = {'f1': f1_score, 'mcc': mcc, 'roc': rocAuc_score, 'aupr': aupr_score, 'accuracy': accuracy, 'auc': auc_score}
    outputEval = {}

    if type(metrics_) == str:
        if metrics_.lower() not in metricsDict:
            raise ValueError(f"{metrics_.lower()} is not supported! f1, mcc, roc, aupr, accuracy, and auc are supported evaluation metrics")
        else:
            outputEval[metrics_.lower()] = metricsDict[metrics_.lower()](y_true, y_probs)
        return outputEval

    elif type(metrics_) == list:
        metricNames = [x.lower() for x in metrics_]
        for metricName in metricNames:
            if metricName not in metricsDict:
                raise ValueError(f"{metricName} is not supported! f1, mcc, roc, aupr, accuracy, and auc are supported evaluation metrics")
            else:
                outputEval[metricName] = metricsDict[metricName](y_true, y_probs)
        return outputEval

    else:
        raise TypeError("metrics must be a string or list of strings")


In [44]:
y_test = np.asarray(y_test)
y_test = (y_test[:,None] == np.arange(2)).astype(int)
y_test
f1_score = evaluate(y_test.argmax(-1),y_preds,'f1')

mcc = evaluate(y_test.argmax(-1),y_preds,'mcc')
roc = evaluate(y_test.argmax(-1),y_preds,'roc')
aupr = evaluate(y_test.argmax(-1),y_preds,'aupr')


print(f1_score)
print(mcc)
print(roc)
print(aupr)

{'f1': 0.18061674008810572}
{'mcc': 0.025829157350258414}
{'roc': 0.5554008828625453}
{'aupr': 0.5505524605183632}


In [45]:
acc = evaluate(y_test.argmax(-1),y_preds,'accuracy')
acc

{'accuracy': 0.5013404825737265}

In [47]:
auc = evaluate(y_test.argmax(-1),y_preds,'auc')
auc

{'auc': 0.5554008828625453}